# COVID-19 Analysis
Checkout the [webpage with live plots](https://claytonpbarrows.github.io/covid-19/).
And if you're intereested:
- [the github repository](https://github.com/claytonpbarrows/covid-19/).
- [the notebook](https://nbviewer.jupyter.org/github/claytonpbarrows/covid-19/blob/master/covid-19.ipynb).
- And if you want to run the notebook interactively through binder (slow), you can do so [here](https://mybinder.org/v2/gh/claytonpbarrows/covid-19/master/).

*This is a very simple estimate of COVID-19 impacts, and I don't claim to know anything other than how to produce a nice figure and do simple math. __So please don't misinterpret anything here__.*

### Dependencies

In [1]:
using Dates
using PlotlyJS
using DataFrames
using CSV

A simple model for calculating cases based on a constatnt doubling time:

In [2]:
function cases(days::Day, current_cases, doubling_time)
    dbl_period = days/doubling_time
    return current_cases * (2 ^ dbl_period)
end

cases (generic function with 1 method)

Here is a function to generate plotly figures based on the simple virus model parameters.

In [3]:
function plot_covid(days, current_cases, doubling_time = Day(7), hosp_rate = 0.1, death_rate = 0.03)
    td = today()
    dates = td:doubling_time:(td+days)
    case_count = [cases(d-td, current_cases, doubling_time) for d in dates]
    total_cases = scatter(x = dates, y = case_count, name = "Infections")
    hosp = scatter(x = dates, y = case_count.*hosp_rate, name = "Hospitalizations")
    deaths = scatter(x = dates, y = case_count.*death_rate, name = "Deaths")
    plot([total_cases, hosp, deaths])
end

plot_covid (generic function with 4 methods)

## COVID-19 parameters
- According to [this article](https://www.jhsph.edu/news/news-releases/2020/new-study-on-COVID-19-estimates-5-days-for-incubation-period.html), the estimated doubling time of COVID-19 is ~5 days.
- According to [the WHO](https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200306-sitrep-46-covid-19.pdf?sfvrsn=96b04adf_2), the mortality rate of COVID-19 is ~3%.
- I don't know where the 10% hospitalization rate comes from.

JHU has put together some more detailed data, and [HDX has put it in a useful format](https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases).

In [4]:
jh_data = DataFrame(CSV.read(download("https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Confirmed.csv")))

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20
,String⍰,String,Float64,Float64,Int64,Int64
1,missing,Thailand,15.0,101.0,2,3
2,missing,Japan,36.0,138.0,2,1
3,missing,Singapore,1.2833,103.833,0,1
4,missing,Nepal,28.1667,84.25,0,0
5,missing,Malaysia,2.5,112.5,0,0
6,British Columbia,Canada,49.2827,-123.121,0,0
7,New South Wales,Australia,-33.8688,151.209,0,0
8,Victoria,Australia,-37.8136,144.963,0,0
9,Queensland,Australia,-28.0167,153.4,0,0


## Colorado
data processing

In [5]:
co_data = jh_data[.!ismissing.(jh_data[!,Symbol("Province/State")]) .& (jh_data[!,Symbol("Province/State")].=="Colorado"),:]
co_data = melt(co_data,names(co_data)[1:4],variable_name = :Date, value_name =:cases)
co_data.Date = Date.(String.(co_data.Date),"m/d/y") + Year(2000)
recent_cases = co_data[co_data.Date .== maximum(co_data.Date),:cases][1]

┌ Warning: `melt(df::AbstractDataFrame, id_vars; variable_name::Symbol=:variable, value_name::Symbol=:value, view::Bool=false)` is deprecated, use `stack(df, Not(id_vars); variable_name=variable_name, value_name=value_name, view=view)` instead.
│   caller = ip:0x0
└ @ Core :-1


45

Plot the colorado projection using the most recent case count

In [6]:
plot_covid(Day(60), recent_cases, Day(5))

┌ Warning: Accessing `scope.id` is deprecated, use `scopeid(scope)` instead.
│   caller = ip:0x0
└ @ Core :-1


data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with field margin"

## US
data processing

In [7]:
us_data = jh_data[.!ismissing.(jh_data[!,Symbol("Country/Region")]) .& (jh_data[!,Symbol("Country/Region")].=="US"),:]
us_data = us_data[.!occursin.(",", us_data[!,Symbol("Province/State")]), :]
us_data = melt(us_data,names(us_data)[1:4],variable_name = :Date, value_name =:cases)
us_data.Date = Date.(String.(us_data.Date),"m/d/y") + Year(2000)
us_data = aggregate(us_data[!,[:Date,:cases]], :Date, sum)
recent_us_cases = us_data[us_data.Date .== maximum(us_data.Date),:cases_sum][1]

┌ Warning: `melt(df::AbstractDataFrame, id_vars; variable_name::Symbol=:variable, value_name::Symbol=:value, view::Bool=false)` is deprecated, use `stack(df, Not(id_vars); variable_name=variable_name, value_name=value_name, view=view)` instead.
│   caller = ip:0x0
└ @ Core :-1


1663

Plot the US projection based on the most recent case  count

In [8]:
plot_covid(Day(100), recent_us_cases, Day(5))

┌ Warning: Accessing `scope.id` is deprecated, use `scopeid(scope)` instead.
│   caller = ip:0x0
└ @ Core :-1


data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with field margin"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*